# Sparse PCA Simulation

Let $$S_{p\times p}=\alpha[1\ \cdots\ 1\ 0\ \cdots\ 0]^T[1\ \cdots\ 1\ 0\ \cdots\ 0] + \beta I_p$$ and $$(Z_{n\times p})_{i, j}\sim\mathcal{N}(0, 1).$$
We define $$X_{n,p}\equiv Z_{n,p}S_{p,p}.$$

In [1]:
import numpy as np
import torch
import random
from matplotlib import pyplot as plt

In [2]:
p = 50
n = 30
alpha = 0.8
beta = 0.2
half = np.empty((p, 1))
for i in range(p):
    if i < p / 2:
        half[i][0] = 1
    else:
        half[i][0] = 0
S = alpha * half @ half.T + beta * np.identity(p)

Z = np.empty((n, p))
for i in range(n):
    for j in range(p):
        Z[i][j] = np.random.normal(0, 1)
        
X = Z @ S

S, Z, X = torch.from_numpy(S), torch.from_numpy(Z), torch.from_numpy(X)

In [3]:
S.shape, Z.shape, X.shape

(torch.Size([50, 50]), torch.Size([30, 50]), torch.Size([30, 50]))

## Method 1
We want to find $(u, v, w)$ such that 
#### $$(u, v, w)=\underset{(u, v, w)}{\text{argmin}}\ \lVert X-u(v\odot w)^T\rVert_F^2$$
where for any matrix $A$, 
#### $$\lVert A\rVert_F^2 = \sum_{i,j}A_{i,j}^2.$$

In [4]:
u = torch.randn(n, 1, requires_grad=True)
v = torch.randn(p, 1, requires_grad=True)
w = torch.randn(p, 1, requires_grad=True)

In [5]:
def objective_function_1(u, v, w):
    A = X - u @ (v * w).T
    return torch.sum(A * A)

In [6]:
epochs = 10000
step_size = 1e-4
for i in range(epochs):
    objective = objective_function_1(u, v, w)
    if i % 100 == 0:
        print('Epoch ' + str(i) + ": loss = " + str(float(objective)))
    objective.backward()
    with torch.no_grad():
        u -= u.grad * step_size
        v -= v.grad * step_size
        w -= w.grad * step_size
        u.grad.zero_()
        v.grad.zero_()
        w.grad.zero_()
print('Epoch ' + str(epochs) + ": loss = " + str(float(objective_function_1(u, v, w))))

Epoch 0: loss = 9340.006853828141
Epoch 100: loss = 5684.4451262344555
Epoch 200: loss = 1156.293232515135
Epoch 300: loss = 76.13295329702514
Epoch 400: loss = 61.93265962788918
Epoch 500: loss = 61.00041138227286
Epoch 600: loss = 60.60948043873523
Epoch 700: loss = 60.37601925948829
Epoch 800: loss = 60.22104029625878
Epoch 900: loss = 60.11112456848986
Epoch 1000: loss = 60.02961601197364
Epoch 1100: loss = 59.96727766155243
Epoch 1200: loss = 59.91852477052376
Epoch 1300: loss = 59.87971058021062
Epoch 1400: loss = 59.84831807384053
Epoch 1500: loss = 59.822531687423414
Epoch 1600: loss = 59.80101516280337
Epoch 1700: loss = 59.782760970148665
Epoch 1800: loss = 59.76701234652928
Epoch 1900: loss = 59.753205749059056
Epoch 2000: loss = 59.740913936578615
Epoch 2100: loss = 59.729832005457496
Epoch 2200: loss = 59.719730763550295
Epoch 2300: loss = 59.710455849802536
Epoch 2400: loss = 59.7018963997779
Epoch 2500: loss = 59.693981124149545
Epoch 2600: loss = 59.68666443826202
Epoch

In [7]:
print(v * w)

tensor([[ 2.0151e+00],
        [ 2.0660e+00],
        [ 2.0426e+00],
        [ 1.9659e+00],
        [ 2.0037e+00],
        [ 1.9751e+00],
        [ 2.0061e+00],
        [ 2.0433e+00],
        [ 2.0083e+00],
        [ 2.0166e+00],
        [ 2.0150e+00],
        [ 1.9982e+00],
        [ 2.0551e+00],
        [ 2.0451e+00],
        [ 2.0017e+00],
        [ 1.9633e+00],
        [ 1.9882e+00],
        [ 1.9879e+00],
        [ 1.9571e+00],
        [ 2.0042e+00],
        [ 1.9992e+00],
        [ 2.0113e+00],
        [ 1.9861e+00],
        [ 1.9969e+00],
        [ 1.9776e+00],
        [-7.2219e-03],
        [-2.7363e-02],
        [ 1.6200e-02],
        [-1.2656e-02],
        [-3.2145e-02],
        [ 1.2234e-02],
        [ 6.1783e-04],
        [-1.0925e-02],
        [ 1.7339e-02],
        [-1.4584e-03],
        [ 2.8750e-03],
        [-2.5875e-04],
        [ 2.3251e-02],
        [ 4.4086e-03],
        [-4.3658e-02],
        [ 1.1940e-02],
        [-3.6621e-02],
        [-1.7383e-02],
        [ 9

## Method 2
We want to find $(u, v, w)$ such that 
#### $$(u, v, w)=\underset{(u, v, w)}{\text{argmin}}\ \lVert X-Xu(v\odot w)^T\rVert_F^2$$
where for any matrix $A$, 
#### $$\lVert A\rVert_F^2 = \sum_{i,j}A_{i,j}^2.$$

In [8]:
u = torch.randn(p, 1, requires_grad=True)
v = torch.randn(p, 1, requires_grad=True)
w = torch.randn(p, 1, requires_grad=True)

In [9]:
def objective_function_2(u, v, w):
    A = X - (X.double() @ u.double()).double() @ (v * w).T.double()
    return torch.sum(A * A)

In [10]:
epochs = 100000
step_size = 5e-7
for i in range(epochs):
    objective = objective_function_2(u, v, w)
    if i % 1000 == 0:
        print('Epoch ' + str(i) + ": loss = " + str(float(objective)))
    objective.backward()
    with torch.no_grad():
        u -= u.grad * step_size
        v -= v.grad * step_size
        w -= w.grad * step_size
#         if (i == epochs - 1):
#             print(u.grad)
        u.grad.zero_()
        v.grad.zero_()
        w.grad.zero_()
print('Epoch ' + str(epochs) + ": loss = " + str(float(objective_function_2(u, v, w))))

Epoch 0: loss = 1016096.311419646
Epoch 1000: loss = 8257.576853699546
Epoch 2000: loss = 7630.172244867054
Epoch 3000: loss = 7318.756262134051
Epoch 4000: loss = 7089.231844062579
Epoch 5000: loss = 6820.800376898386
Epoch 6000: loss = 6397.108471559741
Epoch 7000: loss = 5727.698655353705
Epoch 8000: loss = 4791.779049807653
Epoch 9000: loss = 3646.3175619345366
Epoch 10000: loss = 2497.3930464614114
Epoch 11000: loss = 1637.9070427442666
Epoch 12000: loss = 1119.58438770143
Epoch 13000: loss = 805.0202745665249
Epoch 14000: loss = 570.9062590060433
Epoch 15000: loss = 379.31077816757085
Epoch 16000: loss = 242.0017836530929
Epoch 17000: loss = 168.38658016027344
Epoch 18000: loss = 139.81037323392223
Epoch 19000: loss = 129.19134256971634
Epoch 20000: loss = 123.5860253951027
Epoch 21000: loss = 119.4858670334865
Epoch 22000: loss = 116.08519905550042
Epoch 23000: loss = 113.14648291257757
Epoch 24000: loss = 110.56196425983376
Epoch 25000: loss = 108.25959437251247
Epoch 26000: lo

In [11]:
print(v * w)

tensor([[ 3.7077e-01],
        [ 3.8000e-01],
        [ 3.7584e-01],
        [ 3.6176e-01],
        [ 3.6826e-01],
        [ 3.6338e-01],
        [ 3.6840e-01],
        [ 3.7576e-01],
        [ 3.6944e-01],
        [ 3.7080e-01],
        [ 3.7024e-01],
        [ 3.6751e-01],
        [ 3.7808e-01],
        [ 3.7552e-01],
        [ 3.6837e-01],
        [ 3.6106e-01],
        [ 3.6548e-01],
        [ 3.6508e-01],
        [ 3.5957e-01],
        [ 3.6817e-01],
        [ 3.6797e-01],
        [ 3.6966e-01],
        [ 3.6529e-01],
        [ 3.6703e-01],
        [ 3.6402e-01],
        [-1.2101e-03],
        [-4.8834e-03],
        [ 5.6760e-03],
        [-2.3469e-03],
        [-5.8679e-03],
        [ 2.2008e-03],
        [ 1.4549e-03],
        [-2.1614e-03],
        [ 3.2415e-03],
        [-8.6835e-04],
        [ 5.5093e-04],
        [-2.9431e-05],
        [ 4.4851e-03],
        [ 7.9124e-04],
        [-7.9547e-03],
        [ 2.0991e-03],
        [-6.8081e-03],
        [-3.0611e-03],
        [ 1

## Method 3
We want to find $(u, v, w)$ such that 
#### $$(u, v, w)=\underset{(u, v, w)}{\text{argmin}}\ \lVert X-u(v\odot v - w\odot w)^T\rVert_F^2$$
where for any matrix $A$, 
#### $$\lVert A\rVert_F^2 = \sum_{i,j}A_{i,j}^2.$$

In [12]:
u = torch.randn(n, 1, requires_grad=True)
v = torch.randn(p, 1, requires_grad=True)
w = torch.randn(p, 1, requires_grad=True)

In [13]:
def objective_function_3(u, v, w):
    A = X - u @ (v * v - w * w).T
    return torch.sum(A * A)

In [14]:
epochs = 10000
step_size = 1e-4
for i in range(epochs):
    objective = objective_function_3(u, v, w)
    if i % 100 == 0:
        print('Epoch ' + str(i) + ": loss = " + str(float(objective)))
    objective.backward()
    with torch.no_grad():
        u -= u.grad * step_size
        v -= v.grad * step_size
        w -= w.grad * step_size
        u.grad.zero_()
        v.grad.zero_()
        w.grad.zero_()
print('Epoch ' + str(epochs) + ": loss = " + str(float(objective_function_3(u, v, w))))

Epoch 0: loss = 9350.400290968457
Epoch 100: loss = 6019.834603736181
Epoch 200: loss = 103.60049566826791
Epoch 300: loss = 62.332700579023175
Epoch 400: loss = 60.76330112634478
Epoch 500: loss = 60.2217831291857
Epoch 600: loss = 59.98226087118297
Epoch 700: loss = 59.85953161774421
Epoch 800: loss = 59.79038399869839
Epoch 900: loss = 59.74867687697315
Epoch 1000: loss = 59.722093423809696
Epoch 1100: loss = 59.70430126518195
Epoch 1200: loss = 59.69184482728345
Epoch 1300: loss = 59.68275678141781
Epoch 1400: loss = 59.67588164501595
Epoch 1500: loss = 59.67051188076208
Epoch 1600: loss = 59.66620563445094
Epoch 1700: loss = 59.662676238132875
Epoch 1800: loss = 59.65973043451327
Epoch 1900: loss = 59.65723739487805
Epoch 2000: loss = 59.65509876329735
Epoch 2100: loss = 59.65324698214902
Epoch 2200: loss = 59.651630065917644
Epoch 2300: loss = 59.65020602321553
Epoch 2400: loss = 59.648947557118014
Epoch 2500: loss = 59.647824054482385
Epoch 2600: loss = 59.64681877344373
Epoch 2

In [15]:
print(v * v - w * w)

tensor([[ 2.9115e+00],
        [ 2.9849e+00],
        [ 2.9510e+00],
        [ 2.8403e+00],
        [ 2.8949e+00],
        [ 2.8536e+00],
        [ 2.8983e+00],
        [ 2.9521e+00],
        [ 2.9015e+00],
        [ 2.9135e+00],
        [ 2.9113e+00],
        [ 2.8870e+00],
        [ 2.9692e+00],
        [ 2.9548e+00],
        [ 2.8921e+00],
        [ 2.8365e+00],
        [ 2.8725e+00],
        [ 2.8721e+00],
        [ 2.8275e+00],
        [ 2.8956e+00],
        [ 2.8884e+00],
        [ 2.9059e+00],
        [ 2.8695e+00],
        [ 2.8851e+00],
        [ 2.8573e+00],
        [-1.0429e-02],
        [-3.9532e-02],
        [ 2.3401e-02],
        [-1.8284e-02],
        [-4.6444e-02],
        [ 1.7671e-02],
        [ 1.1991e-02],
        [-1.5781e-02],
        [ 1.4360e-02],
        [-5.9769e-03],
        [ 4.1511e-03],
        [-3.7705e-04],
        [ 3.3591e-02],
        [ 6.3651e-03],
        [-6.3082e-02],
        [ 1.7245e-02],
        [-5.2907e-02],
        [-2.5119e-02],
        [ 1

## Method 4
We want to find $(u, v, w)$ such that 
#### $$(u, v, w)=\underset{(u, v, w)}{\text{argmin}}\ \lVert X-Xu(v\odot v - w\odot w)^T\rVert_F^2$$
where for any matrix $A$, 
#### $$\lVert A\rVert_F^2 = \sum_{i,j}A_{i,j}^2.$$

In [16]:
u = torch.randn(p, 1, requires_grad=True)
v = torch.randn(p, 1, requires_grad=True)
w = torch.randn(p, 1, requires_grad=True)

In [17]:
def objective_function_4(u, v, w):
    A = X - X.double() @ u.double() @ (v * v - w * w).T.double()
    return torch.sum(A * A)

In [18]:
epochs = 100000
step_size = 5e-7
for i in range(epochs):
    objective = objective_function_4(u, v, w)
    if i % 100 == 0:
        print('Epoch ' + str(i) + ": loss = " + str(float(objective)))
    objective.backward()
    with torch.no_grad():
        u -= u.grad * step_size
        v -= v.grad * step_size
        w -= w.grad * step_size
        u.grad.zero_()
        v.grad.zero_()
        w.grad.zero_()
print('Epoch ' + str(epochs) + ": loss = " + str(float(objective_function_4(u, v, w))))

Epoch 0: loss = 750613.6295650415
Epoch 100: loss = 15788.892943445073
Epoch 200: loss = 13173.91828193986
Epoch 300: loss = 11540.745423258704
Epoch 400: loss = 10426.486059073091
Epoch 500: loss = 9613.011791135992
Epoch 600: loss = 8984.86655373474
Epoch 700: loss = 8475.462751428351
Epoch 800: loss = 8043.816956578219
Epoch 900: loss = 7663.4739495748745
Epoch 1000: loss = 7316.839626931993
Epoch 1100: loss = 6992.101207497644
Epoch 1200: loss = 6681.409797841604
Epoch 1300: loss = 6379.716729325701
Epoch 1400: loss = 6083.95043313322
Epoch 1500: loss = 5792.387010233472
Epoch 1600: loss = 5504.199961844033
Epoch 1700: loss = 5219.1467436587145
Epoch 1800: loss = 4937.388735037206
Epoch 1900: loss = 4659.384801606531
Epoch 2000: loss = 4385.8384467317865
Epoch 2100: loss = 4117.635441550416
Epoch 2200: loss = 3855.797498879185
Epoch 2300: loss = 3601.4099650600597
Epoch 2400: loss = 3355.5640539977603
Epoch 2500: loss = 3119.2906595674604
Epoch 2600: loss = 2893.513006657904
Epoch 

In [19]:
print(v * v - w * w)

tensor([[-1.8968e-01],
        [-1.9470e-01],
        [-1.9246e-01],
        [-1.8523e-01],
        [-1.8881e-01],
        [-1.8616e-01],
        [-1.8903e-01],
        [-1.9253e-01],
        [-1.8896e-01],
        [-1.8987e-01],
        [-1.8979e-01],
        [-1.8810e-01],
        [-1.9353e-01],
        [-1.9275e-01],
        [-1.8852e-01],
        [-1.8480e-01],
        [-1.8738e-01],
        [-1.8726e-01],
        [-1.8427e-01],
        [-1.8886e-01],
        [-1.8839e-01],
        [-1.8951e-01],
        [-1.8717e-01],
        [-1.8802e-01],
        [-1.8620e-01],
        [ 7.1323e-04],
        [ 2.6336e-03],
        [-1.4971e-03],
        [ 1.1622e-03],
        [ 3.1174e-03],
        [-1.0930e-03],
        [-8.3965e-04],
        [ 9.5510e-04],
        [-1.6278e-03],
        [ 3.9801e-04],
        [-2.2459e-04],
        [ 1.4306e-04],
        [-2.1572e-03],
        [-5.3418e-04],
        [ 4.0877e-03],
        [-1.0766e-03],
        [ 3.3342e-03],
        [ 1.6348e-03],
        [-7